In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import asyncio
import os
import pandas as pd

from hfppl.llms import CachedCausalLM
from hfppl.inference import smc_standard

from battleship.board import Board, TRIAL_IDS
from battleship.prompting import QuestionGenerationPrompt, TranslationPrompt
from battleship.scoring import compute_score
from battleship.models import QuestionGenerationModel, SingleStepQuestionGenerationModel

In [3]:
# Initialize the HuggingFace model
lm = CachedCausalLM.from_pretrained("codellama/CodeLlama-7b-hf")
# lm = CachedCausalLM.from_pretrained("codellama/CodeLlama-13b-hf")

/home/ubuntu/battleship/.venv/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/ubuntu/battleship/.venv/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/ubuntu/battleship/.venv/lib/python3.11/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


# Run SMC

In [14]:
async def run_smc_baseline(
    n_particles=5,
    trial_ids=TRIAL_IDS,
    board_format="textual",
    n_example_trials=3,
    n_examples_per_trial=3,
    results_file="hfppl_results.csv",
    random_seed=123,
    verbose=False,
):
    results_all = []
    for trial_id in trial_ids:
        print("-" * 80)
        print(f"Trial {trial_id}")
        print("-" * 80)

        # TODO: Sample the question prompt each trial?
        question_prompt = QuestionGenerationPrompt(
            target_trial_id=trial_id,
            board_format=board_format,
            n_example_trials=n_example_trials,
            n_examples_per_trial=n_examples_per_trial,
            include_board=True,
            include_instructions=True,
            # include_board=False,
            # include_instructions=False,
            include_system_prompt=False,
            random_seed=random_seed,
        )

        # TODO: Sample the translation prompt each trial?
        translation_prompt = TranslationPrompt(
            target_trial_id=trial_id,
            n_example_trials=10,
            n_examples_per_trial=1,
            include_system_prompt=False,
            random_seed=random_seed,
        )
        print("-" * 80)
        print("QUESTION PROMPT")
        print("-" * 80)
        print(str(question_prompt))
        print("-" * 80)
        print("TRANSLATION PROMPT")
        print("-" * 80)
        print(str(translation_prompt))
        print("-" * 80)

        lm.clear_cache()
        lm.clear_kv_cache()

        # Significantly speeds up performance, but may result in CUDA out of memory error.
        lm.cache_kv(lm.tokenizer.encode(str(question_prompt)))
        # Additionally, this currently degrades the quality of the translations for an unknown reason.
        # lm.cache_kv(lm.tokenizer.encode(str(translation_prompt)))

        model = QuestionGenerationModel(
            lm=lm,
            board=Board.from_trial_id(trial_id),
            question_prompt=str(question_prompt),
            translation_prompt=str(translation_prompt),
            verbose=verbose,
        )

        particles = await smc_standard(model, n_particles=n_particles)

        results_trial = []
        for i, p in enumerate(particles):
            df_p = pd.DataFrame(p.get_final_results())
            df_p["particle"] = i
            results_trial.append(df_p)
        df_trial = pd.concat(results_trial).reset_index(drop=True)
        df_trial["trial_id"] = trial_id
        results_all.append(df_trial)
        df_results = pd.concat(results_all).reset_index(drop=True)
        df_results.to_csv(results_file, index=False)
    return df_results

In [15]:
# TRIAL_IDS = range(1, 19)
TRIAL_IDS = [13]
RESULTS_FILE = "results/eval_smc.csv"

N_PARTICLES = 3
N_EXAMPLE_TRIALS = 1
N_EXAMPLES_PER_TRIAL = 5
BOARD_FORMAT = "textual"

df_results = await run_smc_baseline(
    n_particles=N_PARTICLES,
    trial_ids=TRIAL_IDS,
    board_format=BOARD_FORMAT,
    n_example_trials=N_EXAMPLE_TRIALS,
    n_examples_per_trial=N_EXAMPLES_PER_TRIAL,
    results_file=RESULTS_FILE,
    verbose=True,
)

--------------------------------------------------------------------------------
Trial 13
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
QUESTION PROMPT
--------------------------------------------------------------------------------
You are playing the board game Battleship. There are three ships on the board: Red, Blue, and Purple. Ships are oriented either horizontally or vertically and can be 2, 3, or 4 tiles in length. The board is a 6x6 grid, with numbered rows 1, 2, 3, 4, 5, 6 and lettered columns A, B, C, D, E, F. Coordinates are specified as a row, column pair. For example, 2-C is the tile in row 2, column C.

You will be given a partially-revealed game board. Your task is to ask a single question that will help you gain information about the position of the remaining hidden ships on the board. You can ask any question, but it must be answerable with a single word

In [12]:
df_results.sort_values("score", ascending=False).head(30)

,prefix,completion,translation,score,type,particle,trial_id
11,At what place is,At what place is the top left part of the purp...,(topleft (coloredTiles Purple)),3.994077,rollout,0,13
54,At what place is,At what place is the top left part of the purp...,(topleft (coloredTiles Purple)),3.994077,rollout,1,13
2,At,At what row and column is the top left part of...,(topleft (coloredTiles Purple)),3.994077,rollout,0,13
45,At,At what row and column is the top left part of...,(topleft (coloredTiles Purple)),3.994077,rollout,1,13
97,At what place is,At what place is the top left part of the purp...,(topleft (coloredTiles Purple)),3.994077,rollout,2,13
98,At what place is,At what place is the purple ship?,(topleft (coloredTiles Purple)),3.994077,rollout,2,13
50,At what place,At what place is the top left of the purple ship?,(topleft (coloredTiles Purple)),3.994077,rollout,1,13
7,At what place,At what place is the top left of the purple ship?,(topleft (coloredTiles Purple)),3.994077,rollout,0,13
93,At what place,At what place is the top left of the purple ship?,(topleft (coloredTiles Purple)),3.994077,rollout,2,13
88,At,At what row and column is the top left part of...,(topleft (coloredTiles Purple)),3.994077,rollout,2,13


In [13]:
df_results.query("type == 'final'")

,prefix,completion,translation,score,type,particle,trial_id
0,At what place is the bottom-left part of the ...,At what place is the bottom-left part of the ...,(bottomleft (coloredTiles Red)),0.0,final,0,13
43,At what place is the bottom-left part of the ...,At what place is the bottom-left part of the ...,(bottomleft (coloredTiles Red)),0.0,final,1,13
86,At what place is the bottom-left tile of the ...,At what place is the bottom-left tile of the ...,(bottomleft (coloredTiles Blue)),0.0,final,2,13


# Debugging

In [ ]:
lm.clear_cache()
lm.clear_kv_cache()

question_prompt = QuestionGenerationPrompt(
    target_trial_id=13,
    board_format="grid",
    n_example_trials=3,
    n_examples_per_trial=3,
    include_board=False,
    include_instructions=False,
    include_system_prompt=False,
    random_seed=123,
)

# TODO: Sample the translation prompt each trial?
translation_prompt = TranslationPrompt(
    target_trial_id=13,
    n_example_trials=10,
    n_examples_per_trial=1,
    include_system_prompt=False,
    random_seed=123,
)

print("-" * 80)
print(str(question_prompt))
print("-" * 80)
print(str(translation_prompt))
print("-" * 80)

lm.cache_kv(lm.tokenizer.encode(str(translation_prompt)))

model = QuestionGenerationModel(
    lm=lm,
    board=Board.from_trial_id(13),
    question_prompt=str(question_prompt),
    translation_prompt=str(translation_prompt),
    verbose=True,
)

partial_questions = [
    "What",
    "Is",
    "How many",
]

print("-" * 80)
print("QUESTIONS")
for q in partial_questions:
    completion = await model._complete_question(q)
    print(completion)


questions = [
    "What is the length of the red ship?",
    "Is there a ship at 3C?",
    "What is the top left corner of the blue ship?",
]

print("-" * 80)
print("TRANSLATIONS")
for q in questions:
    print("-" * 80)
    print(q)
    completion = await model._translate_question(q)
    print(completion)

--------------------------------------------------------------------------------
Question: How many tiles is the red ship?
Question: At what location is the top left part of the red ship?
Question: Is the purple ship 4 tiles long?
Question: Is there a ship at 6A?
Question: How many tiles is the blue ship?
Question: What color is at 4B?
Question: At what location is the top left part of the red ship?
Question: How many tiles is the red ship?
Question: Are all ships 3 tiles long?
Question:
--------------------------------------------------------------------------------
User: At what location is the top left part of the red ship?
Query: (topleft (coloredTiles Red))
User: How many tiles is the purple ship?
Query: (size Purple)
User: Is there a ship at 4A?
Query: (not (== (color 4A) Water))
User: What color is at 6F?
Query: (color 6F)
User: How many tiles is the red ship?
Query: (size Red)
User: How many tiles is the purple ship?
Query: (size Purple)
User: Do the red ship and the purple shi